In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Assignments/marathon_results_2017.xlsx',sheet_name='Cleaned')
df

,Surname,FirstName,Age,AgeDiv,M/F,City,USState,State2,Country,Official Time,Hours,Minutes,Seconds,Total Minutes,TotalSeconds,TimeRange,OverallPos,GenderPos,DivisionPos
0,Kirui,Geoffrey,24,20-24,M,Keringet,NaN,NaN,KEN,02:09:37,2,9,37,130,7777,2-2.5,1,1,1
1,Rupp,Galen,30,30-34,M,Portland,OR,Oregon,USA,02:09:58,2,9,58,130,7798,2-2.5,2,2,2
2,Osako,Suguru,25,25-29,M,Machida-City,NaN,NaN,JPN,02:10:28,2,10,28,130,7828,2-2.5,3,3,3
3,Biwott,Shadrack,32,30-34,M,Mammoth Lakes,CA,California,USA,02:12:08,2,12,8,132,7928,2-2.5,4,4,4
4,Chebet,Wilson,31,30-34,M,Marakwet,NaN,NaN,KEN,02:12:35,2,12,35,133,7955,2-2.5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26405,Steinbach,PaulaEyvonne,61,60-64,F,Ontario,CA,California,USA,07:09:39,7,9,39,430,25779,6.0-7.0,26407,11972,344
26406,Avelino,AndrewR.,25,25-29,M,Fayetteville,NC,North Carolina,USA,07:16:59,7,16,59,437,26219,6.0-7.0,26408,14436,4774
26407,Hantel,Johanna,57,55-59,F,Malvern,PA,Pennsylvania,USA,07:19:37,7,19,37,440,26377,6.0-7.0,26409,11973,698
26408,Reilly,Bill,64,60-64,M,New York,NY,New York,USA,07:20:44,7,20,44,441,26444,6.0-7.0,26410,14437,1043


In [2]:
# prepare table for visualisation

table = df.groupby(['Country','AgeDiv'])['OverallPos'].count()
table = table.to_frame()
table.reset_index(inplace=True)
table.rename(columns = {'Country':"COUNTRY",'AgeDiv':'AGE','OverallPos':'COUNT'}, inplace = True)
# table

In [3]:
# prepare second table for visualisation

table2 = df.groupby(['Country','M/F'])['Total Minutes'].mean()
table2 = table2.to_frame()
table2.reset_index(inplace=True)
table2.rename(columns = {'Country':"COUNTRY",'M/F':'GENDER','Total Minutes':'TIME'}, inplace = True)
# table2

In [4]:
# create drop down list of countries for filter

country_selection = df['Country'].unique().tolist()
country_selection.sort()

In [ ]:
import dash
import dash_table
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input,Output

external_stylesheets = [dbc.themes.LUX]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#FFFFFF',
    'text': '#114B5F'
}

#  App components including graphs & interactive items

app.layout = html.Div(
    style={'backgroundColor': colors['background']}
    ,children=[html.H1(children='Boston Marathon'
              ,style={'textAlign':'center'
                      ,'color':colors['text']
                      ,'padding-top':'3%'
                      ,'padding-bottom':'4%'})
        # Age Division Graph with Country Filter callback
               
      ,html.H3(children='Country Filter:'
              ,style={'color':colors['text']
                      ,'padding-left':'10%'})
      ,dcc.Dropdown(id='country'
                    ,options=[{'label': i, 'value': i} for i in country_selection]
                    ,value='USA'
                    ,style={"width":"40%"
                            ,'padding-bottom':'1%'
                            ,'padding-left':'10%'})
      ,html.Div(id='container'
                ,children=[]
                ,style={'padding-bottom':'1%','padding-left':'10%'})
      ,dcc.Graph(id='ageBar'
                 ,figure={}
                 ,style={'padding-bottom':'4%'
                         ,'padding-left':'10%'})
               
        # Finish time graph - Gender Filter
        ,html.H3(children='Gender Filter:'
                 ,style={'color':colors['text']
                        ,'padding-left':'10%'})
        ,dcc.RadioItems(id='gender'
                        ,options=[{'label': i, 'value': i} for i in ['M', 'F']]
                        ,value='F'
                        ,labelStyle={'padding-bottom':'1%'
                                     ,'padding-left':'10%'})
        ,html.Div(id='container2'
                    ,children=[]
                    ,style={'padding-bottom':'1%'
                            ,'padding-left':'10%'})
        ,dcc.Graph(id='timeBar'
                     ,figure={}
                     ,style={'padding-left':'10%'})
              ]
)

                    
@app.callback(
    [Output(component_id='container',component_property='children'),
    Output(component_id='ageBar',component_property='figure')],
    [Input(component_id='country',component_property='value')])

def update_graph(country_selected):
    dff = table.copy()
    dff = dff[dff['COUNTRY']==country_selected]
    
    container = ""
    
    fig = go.Figure(
        data = [go.Bar(
            x = dff['AGE']
            ,y = dff['COUNT']
            ,textposition = 'auto'
            ,marker = dict(color = '#00b7c2')
            ,hovertemplate =
            '<b>Age: %{x}: </b><br>'+
            'No. Participants:<i> %{y}</i><br>'+
            '<extra></extra>'
            ,name='ageBar',
        )
               ])

    fig.update_layout(
        title='Number of Participants by Age Division<br>' + "{} Age Stats".format(country_selected)
        ,xaxis=dict(title='Age Division'
                    ,showgrid=False
                    ,showticklabels=True)
        ,yaxis=dict(title='# of Runners')
        ,width=800
        ,height=500
        ,legend= {'itemsizing': 'constant'}
        ,font=dict(family='Arial'
                   ,size=12
                   ,color='#7f7f7f'))
    return container,fig 

@app.callback(
    [Output(component_id='container2',component_property='children'),
    Output(component_id='timeBar',component_property='figure')],
    [Input(component_id='gender',component_property='value')])

def update_graph_2(gender_selected):
    dff = table2.copy()
    dff = dff[dff['GENDER']==gender_selected]
    dff =dff.sort_values('TIME',ascending=True)
    
    container2 = ""
    
    fig2 = go.Figure(
        data = [go.Bar(
            x = dff['COUNTRY']
            ,y = dff['TIME']
            ,textposition = 'auto'
            ,marker = dict(color = '#fdcb9e')
            ,hovertemplate =
            '<b>Country: %{x}: </b><br>'+
            'Average Finish Time:<i> %{y}</i><br>'+
            '<extra></extra>'
            ,name='timeBar',
        )
               ])

    fig2.update_layout(
        title='Average Finish Time by Country<br>' + "Gender: {} - Performance".format(gender_selected)
        ,xaxis=dict(title='Country'
                    ,showgrid=False
                    ,showticklabels=True)
        ,yaxis=dict(title='Average Finish Time (mins)')
        ,width=800
        ,height=500
        ,legend= {'itemsizing': 'constant'}
        ,font=dict(family='Arial'
                   ,size=12
                   ,color='#7f7f7f'))
    return container2, fig2

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})
app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
/Users/smiths4/opt/anaconda3/lib/python3.7/site-packages/dash/resources.py:68: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources

127.0.0.1 - - [02/Sep/2020 16:07:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2020 16:07:34] "POST /_dash-update-component HTT